In [ ]:
import pandas as pd
import os
import networkx as nx
import pickle

In [ ]:
PATH = "../data/UMLS/2023AA/META/"
df_conso = pd.read_csv(os.path.join(PATH, 'MRCONSO.tsv'), sep='\t', dtype=str)
df_conso.head()

In [ ]:
df_hier = pd.read_csv(os.path.join(PATH, 'MRHIER.tsv'), sep='\t', dtype=str)
df_hier.head()

In [ ]:
df_conso_sn = df_conso[df_conso['SAB'].isin(["SNOMEDCT_US", "SCTSPA"])]
scui_to_cui_dict = df_conso_sn.groupby('SCUI')['CUI'].agg(lambda x: list(set(x))).to_dict()
aui_to_cui_dict = df_conso_sn.set_index('AUI')['CUI'].to_dict()
df_hier['CUI1'] = df_hier['PAUI'].map(aui_to_cui_dict).dropna()
df_hier.rename(columns={'CUI': 'CUI2'}, inplace=True)
df_parent_child = df_hier[['CUI1', 'CUI2']].dropna().drop_duplicates()
df_parent_child.head()

In [ ]:
df_conso_sn_esp = df_conso_sn[df_conso_sn.LAT == "SPA"]

In [ ]:
grouped = df_conso_sn_esp.groupby('CUI')['STR'].agg(lambda x: list(set([str for str in x if str.strip() != '']))).reset_index()

G = nx.DiGraph()

for index, row in grouped.iterrows():
    G.add_node(row['CUI'], name=row['STR'])

for index, row in df_parent_child.iterrows():
    G.add_edge(row['CUI1'], row['CUI2'])


In [ ]:
with open('scui_to_cui_dict.pkl', 'wb') as f:
    pickle.dump(scui_to_cui_dict, f)

with open('graph_G.pkl', 'wb') as f:
    pickle.dump(G, f)